In [15]:
import pandas as pd
import numpy as np
import folium
import simplejson,requests
import sys
import time
import import_ipynb
import YouTubeAPI as youtube

In [16]:
# 장소 추출 예시를 위한 코드
lex_desc = youtube.ex_desc.split('\n')  # 각 줄마다 split
spl_place = []
for a in lex_desc:                      # 타임라인에 존재하는 장소 추출
    if a.find(":") == 2:
        spl_place.append(a[6:])
    else:
        continue

place = []
for i in spl_place:
    if i == "Intro":                    # Intro는 제거
        continue
    elif "/" in i:
        place.append(i.split(" / ")[0]) # /로 구분된 장소 split
        place.append(i.split(" / ")[1])
    elif "#" in i:
        place.append(i.split(" #")[1])  # # 뒤에 존재하는 장소 추출
    elif "와 "in i:
        place.append(i.split("와 ")[0]) # 와 뒤에 존재하는 장소 추출
        place.append(i.split("와 ")[1])
    else:
        place.append(i)
print(place)


['#국내여행베스트 #대구근교가볼만한곳 #경북여행 #주말여행지 #국내여행지추천 ', '', '안녕하세요. 봄비입니다. 이번에 소개할 여행지는 #국내여행지베스트 경북편이에요. 영상을 만들고 싶어 자료를 찾아보던 중에 경북에서 유튜브 공모전한다는 소식을 들었거든요. "그래! #경북 을 준비해보자." 주제 별로 뽑아봤어요. 더 많은 곳이 있는데 5분 만에 다 소개하기엔 어려울 것 같아서 추리고 추렸어요. ', '', '✨ #경북가볼만한곳 ✨', '00:00 Intro', '00:19 영양 자작나무숲 ', '', '00:41 영주무섬다리', '그 외 영주 가볼 만한 곳: 소수서원과 선비촌, 부석사', '01:06 예천 뿅뿅다리', '', '01:20 포항 곤륜산', '01:25 예천 회룡포 전망대', '01:34 포항 내연산 소금강 전망대', '01:53 포항 스페이스워크', '', '02:20 청송 송소고택', '', '02:41 포항 이가리 닻 전망대', '02:44 포항 호미반도', '03:03 울릉해담길', '', '03:15 예천 활체험센터와 삼강문화단지', '03:32 문경 관광사격장과 에코랄라', '', '03:55 포항 장기읍성', '04:10 청도 청도읍성', '04:14 문경 고모산성', '', '✅ Music - Artlist (https://artlist.io/여행작가_봄비-1479365)', '* 위 링크를 통해 가입하시면 2개월 무료 혜택을 받을 수 있습니다. ', '✅ 자막 #나눔템플릿  #라니자막 ', '', '💜 봄비의 인스타 : http://www.instagram.com/iamhappyy', '💜 자세한 정보는 블로그: http://blog.naver.com/springlll8']
['영양 자작나무숲 ', '영주무섬다리', '예천 뿅뿅다리', '포항 곤륜산', '예천 회룡포 전망대', '포항 내연산 소금강 전망대', '포항 스페이스워크', '청송 송소고택', '포항 이가리 닻 전망대', '포항 호미반도', '울릉해담길', '예천 활체

In [17]:
place_id = []
place_do =  []
place_si =[]
place_name = []
place_address = []
place_phone = []
place_url = []
place_X = []
place_Y = []
kakao_df = pd.DataFrame(columns=['ID','이름','주소','도', '시',
                                 '전화번호','URL','X좌표','Y좌표'])

url = "https://dapi.kakao.com/v2/local/search/keyword.json?" # kakaomap api에 접근
apikey = "c533ba443ba909af8d277567163d03fe"
for i in range(len(place)):    
    query = str(place[i])
    r = requests.get( url, params = {'query':query}, 
                     headers={'Authorization' : 'KakaoAK ' + apikey } )
    js = simplejson.JSONEncoder().encode(r.json())
    
    # 장소를 입력하여 json파일 형태의 데이터 생성
    rj = r.json()                       
    
    # 입력했을 때 결과가 없으면 생략
    if rj['documents'] == []:                        
        continue
    place_id.append(rj['documents'][0]['id'])         # 이후 list에 결과 append
    if rj['meta']['same_name']['selected_region'] == '':
        place_do.append(rj['documents'][0]['address_name'].split()[0])
        place_si.append(rj['documents'][0]['address_name'].split()[1])
    else:
        place_do.append(rj['meta']['same_name']['selected_region'].split()[0])
        place_si.append(rj['meta']['same_name']['selected_region'].split()[1])
    place_name.append(rj['documents'][0]['place_name'])
    place_address.append(rj['documents'][0]['address_name'])
    place_phone.append(rj['documents'][0]['phone'])
    place_url.append(rj['documents'][0]['place_url'])
    place_X.append(rj['documents'][0]['x'])
    place_Y.append(rj['documents'][0]['y'])

{'documents': [{'address_name': '경북 영양군 수비면 죽파리 산 39-1', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 숲', 'distance': '', 'id': '1726199741', 'phone': '', 'place_name': '영양 자작나무숲', 'place_url': 'http://place.map.kakao.com/1726199741', 'road_address_name': '', 'x': '129.26464356321975', 'y': '36.742549456062'}], 'meta': {'is_end': True, 'pageable_count': 1, 'same_name': {'keyword': '자작나무숲', 'region': [], 'selected_region': '경북 영양군'}, 'total_count': 1}}
{'documents': [{'address_name': '경북 영주시 문수면 탄산리 766', 'category_group_code': '', 'category_group_name': '', 'category_name': '교통,수송 > 도로시설 > 교량,다리', 'distance': '', 'id': '11623349', 'phone': '', 'place_name': '외나무다리', 'place_url': 'http://place.map.kakao.com/11623349', 'road_address_name': '경북 영주시 문수면 무섬로234번길 41', 'x': '128.61942389375', 'y': '36.7300208222057'}], 'meta': {'is_end': True, 'pageable_count': 1, 'same_name': {'keyword': '영주무섬다리', 'region': [], 'selected_region': ''}, 'total_co

{'documents': [{'address_name': '경북 포항시 북구 청하면 이가리 산 67-3', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 전망대', 'distance': '', 'id': '446522573', 'phone': '054-270-3204', 'place_name': '이가리 닻전망대', 'place_url': 'http://place.map.kakao.com/446522573', 'road_address_name': '', 'x': '129.379108982736', 'y': '36.1878660301882'}], 'meta': {'is_end': True, 'pageable_count': 1, 'same_name': {'keyword': '이가리 닻 전망대', 'region': [], 'selected_region': '경북 포항시'}, 'total_count': 1}}
{'documents': [{'address_name': '경북 포항시 남구 동해면 입암리 350', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 도보여행 > 둘레길', 'distance': '', 'id': '213160097', 'phone': '', 'place_name': '호미반도해안둘레길', 'place_url': 'http://place.map.kakao.com/213160097', 'road_address_name': '', 'x': '129.481897009713', 'y': '36.0155580430111'}, {'address_name': '경북 포항시 남구 동해면 임곡리', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name

In [18]:
kakao_df=pd.DataFrame([place_id,place_name,place_address,place_do,place_si,place_phone,place_url,place_X,place_Y]).T
kakao_df.columns=['placeID','pname','paddress',
                  'pdo', 'psi','pphone','pdaumURL','pX','pY']
kakao_df                                              # 데이터프레임 생성

,placeID,pname,paddress,pdo,psi,pphone,pdaumURL,pX,pY
0,1726199741,영양 자작나무숲,경북 영양군 수비면 죽파리 산 39-1,경북,영양군,,http://place.map.kakao.com/1726199741,129.26464356321975,36.742549456062
1,11623349,외나무다리,경북 영주시 문수면 탄산리 766,경북,영주시,,http://place.map.kakao.com/11623349,128.61942389375,36.7300208222057
2,20263100,뿅뿅다리쉼터,경북 예천군 용궁면 대은리 349,경북,예천군,,http://place.map.kakao.com/20263100,128.327065979419,36.5794683277883
3,25178704,곤륜산,경북 포항시 북구 흥해읍 칠포리 산 85-1,경북,포항시,,http://place.map.kakao.com/25178704,129.39158115005458,36.13469443760141
4,20035376,회룡대,경북 예천군 지보면 마산리 산 113-1,경북,예천군,054-650-6395,http://place.map.kakao.com/20035376,128.315197354288,36.5782050189438
5,1440340624,환호공원 스페이스워크,경북 포항시 북구 두호동 산 8,경북,포항시,054-270-8282,http://place.map.kakao.com/1440340624,129.390266451304,36.0650661624599
6,27149596,송소고택,경북 청송군 파천면 덕천리 176,경북,청송군,054-874-6556,http://place.map.kakao.com/27149596,129.02640558673312,36.436239232964596
7,446522573,이가리 닻전망대,경북 포항시 북구 청하면 이가리 산 67-3,경북,포항시,054-270-3204,http://place.map.kakao.com/446522573,129.379108982736,36.1878660301882
8,213160097,호미반도해안둘레길,경북 포항시 남구 동해면 입암리 350,경북,포항시,,http://place.map.kakao.com/213160097,129.481897009713,36.0155580430111
9,36690600,활기찬 예천활체험센터,경북 예천군 예천읍 청복리 467-1,경북,예천군,054-653-2434,http://place.map.kakao.com/36690600,128.46674078386096,36.636595448945954
